In [1]:
%matplotlib nbagg

import numpy as np
import matplotlib.pyplot as plt
from math import *
import matplotlib.animation as animation

# ヤコビ行列の逆行列を求める
def calc_yacobian_inv(phi1, phi2, phi3, l1, l2, l3):
    yacobian = np.array([[-l1*sin(phi1) - l2*sin(phi1+phi2) - l3*sin(phi1+phi2+phi3),-l2*sin(phi1+phi2) - l3*sin(phi1+phi2+phi3),-l3*sin(phi1+phi2+phi3)],
                         [l1*cos(phi1) + l2*cos(phi1+phi2) + l3*cos(phi1+phi2+phi3),l2*cos(phi1+phi2) + l3*cos(phi1+phi2+phi3),l3*cos(phi1+phi2+phi3)],
                         [1,1,1]])
    try:
        yacobian_inv = np.linalg.inv(yacobian) #ヤコビ行列の逆行列を求める
    except:
        yacobian_inv = np.linalg.pinv(yacobian) #正則で求められないとき疑似逆行列を求める

    return yacobian_inv

# 追従させる軌道 直線 端点の2つの座標を与える.  binは時間の分割数. 各時刻における所望の座標のリストを返す(indexは分割数個)
def calc_line_coodinate(start,end,bin):
    x1 = start[0]
    y1 = start[1]
    x2 = end[0]
    y2 = end[1]

    x_refs = []
    y_refs = []
    for i in range(bin+1):
        x = (x2-x1)*(i/bin) + x1
        y = (y2-y1)*(i/bin) + y1
        x_refs.append(x)
        y_refs.append(y)

    return x_refs, y_refs

# 追従させる軌道 直線 端点の2つの座標を与える.  req_tは処理時間. 各時刻における所望の座標のリストを返す(indexは分割数個)
def calc_line_coodinate_bytime(start, end, req_t):
    cur_t = 0
    x_refs = []
    y_refs = []
    while(cur_t < req_t):
        s = 6*(cur_t/req_t)**5 - 15*(cur_t/req_t)**4 + 10*(cur_t/req_t)**3
        x_ref = start[0]*(1-s) + end[0]*s
        y_ref = start[1]*(1-s) + end[1]*s
        x_refs.append(x_ref)
        y_refs.append(y_ref)
        cur_t += 1
    
    return x_refs, y_refs

# 追従させる軌道 円 始点と回転角を与える
def calc_circle_coodinate(start,theta,bin):
    x_refs = []
    y_refs = []
    x1 = start[0]
    y1 = start[1]
    x_refs.append(x1)
    y_refs.append(y1)
    for i in range(bin+1):
        # 回転行列を計算
        r_mat = np.array([[cos(theta/bin),-sin(theta/bin)],
                          [sin(theta/bin), cos(theta/bin)]])
        
        x = np.array([[x1],
                      [y1]])
        
        x_new = np.dot(r_mat, x)
        x1 = x_new[0][0]
        y1 = x_new[1][0]
        x_refs.append(x1)
        y_refs.append(y1)
    
    return x_refs, y_refs
    
# 順運動学で第一関節の位置を計算
def calc_cur_first_pos(l1,phi1):
    x = l1*cos(phi1)
    y = l1*sin(phi1)
    cur_first_pos = [x,y]

    return cur_first_pos

# 順運動学で第二関節の位置を計算
def calc_cur_second_pos(l1,l2,phi1,phi2):
    x = l1*cos(phi1) + l2*cos(phi1+phi2)
    y = l1*sin(phi1) + l2*sin(phi1+phi2)
    cur_second_pos = [x,y]

    return cur_second_pos

# 順運動学で現在の関節角度から手先位置を計算
def calc_cur_hand_pos(l1,l2,l3,phi1,phi2,phi3):
    x = l1*cos(phi1) + l2*cos(phi1+phi2) + l3*cos(phi1+phi2+phi3)
    y = l1*sin(phi1) + l2*sin(phi1+phi2) + l3*sin(phi1+phi2+phi3)
    alpha = phi1+phi2+phi3

    cur_hand_pos = np.array([[x],
                             [y],
                             [alpha]])

    return cur_hand_pos

# 加える関節角度Φ1,Φ2,Φ3を求める cur_posは3×1行列 [x,y,alpha] (numpy_array)
def calc_added_angle(cur_pos, x_refs, y_refs, index, yacobian_inv):
    goal_pos = np.array([[x_refs[index]],
                         [y_refs[index]],
                         [0]]) #αは0とする
    dif_pos = goal_pos - cur_pos # 現在位置との差
    #print(goal_pos)
    #print(cur_pos)
    #print(dif_pos)
    added_angles = np.dot(yacobian_inv, dif_pos) # 加えるべき関節角度の配列

    return added_angles

In [ ]:
# params
phi1 = 0.2
phi2 = 0.1
phi3 = -0.3

# リンクの長さ
l1 = 83
l2 = 93.5
l3 = 120

# 初期位置
cur_hand_pos = calc_cur_hand_pos(l1,l2,l3,phi1,phi2,phi3)

# 円軌道用
req_t = 30000
theta = -pi/4

# 直線軌道用
start = [cur_hand_pos[0][0], cur_hand_pos[1][0]]
end = [cur_hand_pos[0][0], cur_hand_pos[1][0] - 150]
#x_refs, y_refs = calc_line_coodinate(start,end,req_t)
x_refs, y_refs = calc_line_coodinate_bytime(start,end,req_t)
#x_refs, y_refs = calc_circle_coodinate(start,theta,req_t)

# 描画用
ims = []
fig = plt.figure()
flag_legend = True

# iteration
for i in range(0,req_t):
    print("iter :" +str(i))
    # 各関節の現在位置を計算
    cur_first_pos = calc_cur_first_pos(l1,phi1)
    cur_second_pos = calc_cur_second_pos(l1,l2,phi1,phi2)
    cur_hand_pos = calc_cur_hand_pos(l1,l2,l3,phi1,phi2,phi3)
    #print("x_dif : " + str(x_refs[i]-cur_hand_pos[0][0])) 
    #print("y_dif : " + str(y_refs[i]-cur_hand_pos[1][0]))
    
    # 目標値との誤差  適当に初期化
    error1 = 6
    error2 = 6
    j = 0
    
    while (abs(error1) > 5 or abs(error2) > 5):
        # ヤコビ逆行列を計算し、関節を追加する
        yacobian_inv = calc_yacobian_inv(l1,l2,l3,phi1,phi2,phi3)
        added_angles = calc_added_angle(cur_hand_pos, x_refs, y_refs, i, yacobian_inv)
        
        # チューニングゲイン
        K = 1
        phi1 += K*added_angles[0][0]
        phi2 += K*added_angles[1][0]
        phi3 += K*added_angles[2][0]
        
        cur_hand_pos = calc_cur_hand_pos(l1,l2,l3,phi1,phi2,phi3)
        error1 = x_refs[i] - cur_hand_pos[0][0]
        error2 = y_refs[i] - cur_hand_pos[1][0]
        j += 1
        """
        if j < 10:
            break
            print("x_dif : " + str(x_refs[i]-cur_hand_pos[0][0])) 
            print("y_dif : " + str(y_refs[i]-cur_hand_pos[1][0]))
            print("a_dif : " + str(0-cur_hand_pos[2][0]))
        """
    
    #プロット用 遅くなるので10回に1回表示
    if i%5 == 0:
        im = plt.plot([0,cur_first_pos[0],cur_second_pos[0],cur_hand_pos[0][0]],[0,cur_first_pos[1],cur_second_pos[1],cur_hand_pos[1][0]],'b-o',label="robot arm",color="m")
        ims.append(im)
    if flag_legend: # 一回のみ凡例を描画
        plt.plot(x_refs,y_refs,label="target trajectory")
        #plt.ylim(-2,2)
        plt.xlim(-100,300)
        plt.axes().set_aspect('equal')
        plt.legend()
        flag_legend = False


ani = animation.ArtistAnimation(fig, ims, interval=1)
#ani.save('yacobi_line_compliance_3link_02.gif', writer='pillow', fps=50)
fig.show()
print("done")

<IPython.core.display.Javascript object>

iter :0
iter :1
iter :2
iter :3
iter :4
iter :5
iter :6
iter :7
iter :8
iter :9
iter :10
iter :11
iter :12
iter :13
iter :14
iter :15
iter :16
iter :17
iter :18
iter :19
iter :20
iter :21
iter :22
iter :23
iter :24
iter :25
iter :26
iter :27
iter :28
iter :29
iter :30
iter :31
iter :32
iter :33
iter :34
iter :35
iter :36
iter :37
iter :38
iter :39
iter :40
iter :41
iter :42
iter :43
iter :44
iter :45
iter :46
iter :47
iter :48
iter :49
iter :50
iter :51
iter :52
iter :53
iter :54
iter :55
iter :56
iter :57
iter :58
iter :59
iter :60
iter :61
iter :62
iter :63
iter :64
iter :65
iter :66
iter :67
iter :68
iter :69
iter :70
iter :71
iter :72
iter :73
iter :74
iter :75
iter :76
iter :77
iter :78
iter :79
iter :80
iter :81
iter :82
iter :83
iter :84
iter :85
iter :86
iter :87
iter :88
iter :89
iter :90
iter :91
iter :92
iter :93
iter :94
iter :95
iter :96
iter :97
iter :98
iter :99
iter :100
iter :101
iter :102
iter :103
iter :104
iter :105
iter :106
iter :107
iter :108
iter :109
iter :110


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


iter :171
iter :172
iter :173
iter :174
iter :175
iter :176
iter :177
iter :178
iter :179
iter :180
iter :181
iter :182
iter :183
iter :184
iter :185
iter :186
iter :187
iter :188
iter :189
iter :190
iter :191
iter :192
iter :193
iter :194
iter :195
iter :196
iter :197
iter :198
iter :199
iter :200
iter :201
iter :202
iter :203
iter :204
iter :205
iter :206
iter :207
iter :208
iter :209
iter :210
iter :211
iter :212
iter :213
iter :214
iter :215
iter :216
iter :217
iter :218
iter :219
iter :220
iter :221
iter :222
iter :223
iter :224
iter :225
iter :226
iter :227
iter :228
iter :229
iter :230
iter :231
iter :232
iter :233
iter :234
iter :235
iter :236
iter :237
iter :238
iter :239
iter :240
iter :241
iter :242
iter :243
iter :244
iter :245
iter :246
iter :247
iter :248
iter :249
iter :250
iter :251
iter :252
iter :253
iter :254
iter :255
iter :256
iter :257
iter :258
iter :259
iter :260
iter :261
iter :262
iter :263
iter :264
iter :265
iter :266
iter :267
iter :268
iter :269
iter :270


iter :1131
iter :1132
iter :1133
iter :1134
iter :1135
iter :1136
iter :1137
iter :1138
iter :1139
iter :1140
iter :1141
iter :1142
iter :1143
iter :1144
iter :1145
iter :1146
iter :1147
iter :1148
iter :1149
iter :1150
iter :1151
iter :1152
iter :1153
iter :1154
iter :1155
iter :1156
iter :1157
iter :1158
iter :1159
iter :1160
iter :1161
iter :1162
iter :1163
iter :1164
iter :1165
iter :1166
iter :1167
iter :1168
iter :1169
iter :1170
iter :1171
iter :1172
iter :1173
iter :1174
iter :1175
iter :1176
iter :1177
iter :1178
iter :1179
iter :1180
iter :1181
iter :1182
iter :1183
iter :1184
iter :1185
iter :1186
iter :1187
iter :1188
iter :1189
iter :1190
iter :1191
iter :1192
iter :1193
iter :1194
iter :1195
iter :1196
iter :1197
iter :1198
iter :1199
iter :1200
iter :1201
iter :1202
iter :1203
iter :1204
iter :1205
iter :1206
iter :1207
iter :1208
iter :1209
iter :1210
iter :1211
iter :1212
iter :1213
iter :1214
iter :1215
iter :1216
iter :1217
iter :1218
iter :1219
iter :1220
iter :1221

...

iter :15351
iter :15352
iter :15353
iter :15354
iter :15355
iter :15356
iter :15357
iter :15358
iter :15359
iter :15360
iter :15361
iter :15362
iter :15363
iter :15364
iter :15365
iter :15366
iter :15367
iter :15368
iter :15369
iter :15370
iter :15371
iter :15372
iter :15373
iter :15374
iter :15375
iter :15376
iter :15377
iter :15378
iter :15379
iter :15380
iter :15381
iter :15382
iter :15383
iter :15384
iter :15385
iter :15386
iter :15387
iter :15388
iter :15389
iter :15390
iter :15391
iter :15392
iter :15393
iter :15394
iter :15395
iter :15396
iter :15397
iter :15398
iter :15399
iter :15400
iter :15401
iter :15402
iter :15403
iter :15404
iter :15405
iter :15406
iter :15407
iter :15408
iter :15409
iter :15410
iter :15411
iter :15412
iter :15413
iter :15414
iter :15415
iter :15416
iter :15417
iter :15418
iter :15419
iter :15420
iter :15421
iter :15422
iter :15423
iter :15424
iter :15425
iter :15426
iter :15427
iter :15428
iter :15429
iter :15430
iter :15431
iter :15432
iter :15433
iter

iter :16137
iter :16138
iter :16139
iter :16140
iter :16141
iter :16142
iter :16143
iter :16144
iter :16145
iter :16146
iter :16147
iter :16148
iter :16149
iter :16150
iter :16151
iter :16152
iter :16153
iter :16154
iter :16155
iter :16156
iter :16157
iter :16158
iter :16159
iter :16160
iter :16161
iter :16162
iter :16163
iter :16164
iter :16165
iter :16166
iter :16167
iter :16168
iter :16169
iter :16170
iter :16171
iter :16172
iter :16173
iter :16174
iter :16175
iter :16176
iter :16177
iter :16178
iter :16179
iter :16180
iter :16181
iter :16182
iter :16183
iter :16184
iter :16185
iter :16186
iter :16187
iter :16188
iter :16189
iter :16190
iter :16191
iter :16192
iter :16193
iter :16194
iter :16195
iter :16196
iter :16197
iter :16198
iter :16199
iter :16200
iter :16201
iter :16202
iter :16203
iter :16204
iter :16205
iter :16206
iter :16207
iter :16208
iter :16209
iter :16210
iter :16211
iter :16212
iter :16213
iter :16214
iter :16215
iter :16216
iter :16217
iter :16218
iter :16219
iter

iter :16911
iter :16912
iter :16913
iter :16914
iter :16915
iter :16916
iter :16917
iter :16918
iter :16919
iter :16920
iter :16921
iter :16922
iter :16923
iter :16924
iter :16925
iter :16926
iter :16927
iter :16928
iter :16929
iter :16930
iter :16931
iter :16932
iter :16933
iter :16934
iter :16935
iter :16936
iter :16937
iter :16938
iter :16939
iter :16940
iter :16941
iter :16942
iter :16943
iter :16944
iter :16945
iter :16946
iter :16947
iter :16948
iter :16949
iter :16950
iter :16951
iter :16952
iter :16953
iter :16954
iter :16955
iter :16956
iter :16957
iter :16958
iter :16959
iter :16960
iter :16961
iter :16962
iter :16963
iter :16964
iter :16965
iter :16966
iter :16967
iter :16968
iter :16969
iter :16970
iter :16971
iter :16972
iter :16973
iter :16974
iter :16975
iter :16976
iter :16977
iter :16978
iter :16979
iter :16980
iter :16981
iter :16982
iter :16983
iter :16984
iter :16985
iter :16986
iter :16987
iter :16988
iter :16989
iter :16990
iter :16991
iter :16992
iter :16993
iter

iter :17777
iter :17778
iter :17779
iter :17780
iter :17781
iter :17782
iter :17783
iter :17784
iter :17785
iter :17786
iter :17787
iter :17788
iter :17789
iter :17790
iter :17791
iter :17792
iter :17793
iter :17794
iter :17795
iter :17796
iter :17797
iter :17798
iter :17799
iter :17800
iter :17801
iter :17802
iter :17803
iter :17804
iter :17805
iter :17806
iter :17807
iter :17808
iter :17809
iter :17810
iter :17811
iter :17812
iter :17813
iter :17814
iter :17815
iter :17816
iter :17817
iter :17818
iter :17819
iter :17820
iter :17821
iter :17822
iter :17823
iter :17824
iter :17825
iter :17826
iter :17827
iter :17828
iter :17829
iter :17830
iter :17831
iter :17832
iter :17833
iter :17834
iter :17835
iter :17836
iter :17837
iter :17838
iter :17839
iter :17840
iter :17841
iter :17842
iter :17843
iter :17844
iter :17845
iter :17846
iter :17847
iter :17848
iter :17849
iter :17850
iter :17851
iter :17852
iter :17853
iter :17854
iter :17855
iter :17856
iter :17857
iter :17858
iter :17859
iter

iter :18511
iter :18512
iter :18513
iter :18514
iter :18515
iter :18516
iter :18517
iter :18518
iter :18519
iter :18520
iter :18521
iter :18522
iter :18523
iter :18524
iter :18525
iter :18526
iter :18527
iter :18528
iter :18529
iter :18530
iter :18531
iter :18532
iter :18533
iter :18534
iter :18535
iter :18536
iter :18537
iter :18538
iter :18539
iter :18540
iter :18541
iter :18542
iter :18543
iter :18544
iter :18545
iter :18546
iter :18547
iter :18548
iter :18549
iter :18550
iter :18551
iter :18552
iter :18553
iter :18554
iter :18555
iter :18556
iter :18557
iter :18558
iter :18559
iter :18560
iter :18561
iter :18562
iter :18563
iter :18564
iter :18565
iter :18566
iter :18567
iter :18568
iter :18569
iter :18570
iter :18571
iter :18572
iter :18573
iter :18574
iter :18575
iter :18576
iter :18577
iter :18578
iter :18579
iter :18580
iter :18581
iter :18582
iter :18583
iter :18584
iter :18585
iter :18586
iter :18587
iter :18588
iter :18589
iter :18590
iter :18591
iter :18592
iter :18593
iter

In [2]:
x = -3
abs(x)

3